In [ ]:

#------------------------------------------------------------------------
# WES 207 Demo Code
#------------------------------------------------------------------------

from pynq import Overlay
from pynq import MMIO
import time
import numpy as np

print('done')

In [ ]:

# Load bitstream
ol = Overlay('AXI_SPI_BUS_PMOD_NAV_31.bit')

# Download bitstream to FPGA
t_before_bitstream = time.time()
ol.download()
t_after_bitstream = time.time()

print('done')
print('  ')
print(t_after_bitstream - t_before_bitstream, 'seconds to program bitstream')

### initializing MMIO objects for IP core debug

In [ ]:
# new core test

# testing SPI output

# SPI CORE config
QUAD_SPI_BASE_ADDERSS = 0x41E00000
SPI_DRIVER_BASE_ADDRESS = 0x40000000
SPI_DRIVER_DATA_ADDRESS = 0x40004000
#SPI_DRIVER_TEST_ADDRESS = 0x40012000

#POSITION_CTRL_DATA_ADDRESS = 0x40010000
#POSITION_TEST_DATA_ADDRESS = 0x40020000
ADDRESS_RANGE = 0x1000 # 4k size
#ADDRESS_RANGE_8K = 0x2000 # 8k size
ADDRESS_RANGE_16K = 0x4000 # 16k size

# creating SPI CORE object for messages
QSPI_CORE = MMIO(QUAD_SPI_BASE_ADDERSS,ADDRESS_RANGE, True)
SPI_DRIVER = MMIO(SPI_DRIVER_BASE_ADDRESS,ADDRESS_RANGE, True)
SPI_DATA = MMIO(SPI_DRIVER_DATA_ADDRESS,ADDRESS_RANGE_16K,True)
#SPI_TEST = MMIO(SPI_DRIVER_TEST_ADDRESS,ADDRESS_RANGE_8K,True)

#POS_DATA = MMIO(POSITION_CTRL_DATA_ADDRESS, ADDRESS_RANGE,True)
#POS_TEST = MMIO(POSITION_TEST_DATA_ADDRESS, ADDRESS_RANGE,True)

print(' ')
print('done')

## Debug Cells Below

### reading CTRL register from HLS core

In [ ]:
# test reads

test = SPI_DRIVER.read(0x00)
print('\nCalling SPI_Driver.read(0x00):')
print(test)

print('done')

### writing ap_start and auto_restart CTRL bits in HLS core

In [ ]:
# test code

# setting ap_start/auto reset on SPI Driver
#SPI_DRIVER.write(0x00,0x81) #runs continuously, see xax_spi_driver_hw.h

# setting 
SPI_DRIVER.write(0x00,0x01) #runs once
#SPI_DRIVER.write(0x00,0x81) #runs continuously, see xax_spi_driver_hw.h

READ = SPI_DRIVER.read(0x00)
print('\nCalling SPI_DRIVER.write(0x00,0x01)')
print('Calling SPI_Driver.read(0x00):')
print(READ)

print('done')

### test read of QUAD SPI IP core configuration registers

In [ ]:
# this is a basic configuration of the SPI core
# I would read through the docs a bit more and make sure this is correct
# it seems to work as is, 8 bit addres and 8 bit data with SS and CLK

# testing config
QSPI_CORE.write(0x60, 0x6) # enable QSPI core in master mode, auto SS
QSPI_CORE.write(0x70, 0xFFFE) # enable SS 0

# reading back config registers
read1 = QSPI_CORE.read(0x60)
read2 = QSPI_CORE.read(0x70)

print(read1)
print(read2)
print('done')

### Testing read from PMODNav GYRO/ACC WHO_AM_I Register 0x0F

In [ ]:
# PMODNav setup
WRITE_OFFSET = 0x68 #SPI_DTR, SPI data transmit
CTRL_REG5_XL = 0x38
CTRL_REG6_XL = 0xDB
WRITE_CFG  = 0x7F
READ_CFG = 1<<7
CTRL_REG1_G = 0x6F
CTRL_REG4 = 0x38

ADDRESS = CTRL_REG5_XL & WRITE_CFG
DATA = 0x38 # enable all 3 GYRO axis
MESSAGE = (ADDRESS << 8) | DATA

QSPI_CORE.write(WRITE_OFFSET,MESSAGE)

ADDRESS = CTRL_REG6_XL & WRITE_CFG
DATA = 0xC0  # set GYRO odr to 952 Hz
MESSAGE = (ADDRESS << 8) | DATA

QSPI_CORE.write(WRITE_OFFSET,MESSAGE)

ADDRESS = CTRL_REG1_G & WRITE_CFG
DATA = 0xC0
MESSAGE = (ADDRESS << 8) | DATA

QSPI_CORE.write(WRITE_OFFSET,MESSAGE)

ADDRESS = CTRL_REG4 & WRITE_CFG
DATA = 0x38
MESSAGE = (ADDRESS << 8) | DATA

QSPI_CORE.write(WRITE_OFFSET,MESSAGE)

In [ ]:
# testing read WHO_AM_I
GYRO_WHO_AM_I = 0x0F

# writing read command to PMODNav WHO_AM_I register 0x0F
WRITE_OFFSET = 0x68
ADDRESS = GYRO_WHO_AM_I | READ_CFG
DATA = 0x00
MESSAGE = (ADDRESS << 8) | DATA ### Testing read from SPI Core
QSPI_CORE.write(WRITE_OFFSET, MESSAGE) # try to send data

# reading response from PMODNav from SPI core
READ_OFFSET = 0x6C #SPI_DRR, SPI data reveive

r1 = QSPI_CORE.read(READ_OFFSET)
#r2 = SPI_CORE.read(0x7400)

print('offset: {0:x}\t'.format(READ_OFFSET) + 'data: {0:x}'.format(r1))
#print('offset: {0:x}\t'.format(0x7400) + 'data: {0:x}'.format(r2))

## testing read from SPI_DATA and SPI_TEST

In [ ]:
###### pmod_data is 32 bits
index_0 = 0x00
index_1 = 0x04
index_2 = 0x08
index_3 = 0x0C
index_4 = 0x10
index_5 = 0x14
index_6 = 0x18
index_7 = 0x1C
index_8 = 0x20

TEST_READ0 = QSPI_CORE.read(0x78)
print(TEST_READ0)

#TEST_READ = QSPI_CORE.read(0x6C)
#print(TEST_READ)

TEST_READ10 = QSPI_CORE.read(0x64)
print(TEST_READ10 & 0x1)

TEST_READ1 = SPI_DATA.read(index_0)# 32 bit version, see HLS
print(TEST_READ1)

#SPI_DATA.write(index_1,69) # 32 bit version, see HLS

TEST_READ2 = SPI_DATA.read(index_1)# 32 bit version, see HLS
print(TEST_READ2)

TEST_READ3 = SPI_DATA.read(index_2)# 32 bit version, see HLS
print(TEST_READ3)

TEST_READ4 = SPI_DATA.read(index_3)# 32 bit version, see HLS
print(TEST_READ4)

TEST_READ5 = SPI_DATA.read(index_4)# 32 bit version, see HLS
print(TEST_READ5)

TEST_READ6 = SPI_DATA.read(index_5)# 32 bit version, see HLS
print(TEST_READ6)

print('\n\n -- DEBUG ONLY, not valid data --')
TEST_READ7 = SPI_DATA.read(index_6)# 32 bit version, see HLS
print(TEST_READ7)

TEST_READ8 = SPI_DATA.read(index_7)# 32 bit version, see HLS
print(TEST_READ8)

TEST_READ9 = SPI_DATA.read(index_8)# 32 bit version, see HLS
print(TEST_READ9)

#TEST_READ3 = SPI_TEST.read(index_1)
#print('data: {0:x}'.format(TEST_READ3))

#TEST_READ4 = SPI_TEST.read(index_2)
#print('data: {0:x}'.format(TEST_READ4))

#572719103d = 2222FFFF, when reading from SPI_TEST.read(index_0)
#1145320243 = 44443333, when reading from SPI_TEST.read(index_1)

#TEST_READ2 = POS_DATA.read(index_1)
#print(TEST_READ2)
#TEST_READ3 = POS_TEST.read(index_1)
#print(TEST_READ3)

print(' ')
print('done')

### Testing SPI vein integrated with AXI Quad SPI, AXI_SPI_Driver (custom), and positionCtrl (custom)

In [ ]:

# HLS test write
#UART_Data.write(0x00,0x45)
SPI_DATA.write(0x0000,0x0045)
POS_DATA.write(0x0000,0x0045)
POS_TEST.write(0x0000,0x0045)

# HLS test read
TEST_READ1 = SPI_DATA.read(0x0000)
TEST_READ2 = POS_DATA.read(0x0000)
TEST_READ3 = POS_TEST.read(0x0000)

print('Test Data: ')
print(TEST_READ1)
print('')
print('Test Data: ')
print(TEST_READ2)
print('')
print('Test Data: ')
print(TEST_READ3)
print('')

print('')
print('done')

In [ ]:


# Address space definitions for PMODNAV
ACC_X_L_ADDR   =  0x28
MAG_X_L_ADDR   =  0x28
GYRO_X_L_ADDR  =  0x18
ALT_X_L_ADDR   =  0x28

CTRL_REG6_XL_ADDR  =  0x20

X_DIR_SEL    =   0x01
Y_DIR_SEL    =   0x02
Z_DIR_SEL    =   0x03

READ_CFG    =   0x80  # bit 7 high
WRITE_CFG   =   0x7F  # bit 7 low
MS_CFG      =   0xC0  # 11000000

STATE_ON    =   0x01
STATE_OFF   =   0x00

CTRL_REG5_XL =  0x1F
CTRL_REG6_XL =  0x20
CTRL_REG4    =  0x1E

CTRL_REG1_M  =  0x20
CTRL_REG2_M  =  0x21
CTRL_REG3_M  =  0x22
CTRL_REG4_M  =  0x23
CTRL_REG5_M  =  0x24

CTRL_REG1_G  =  0x10
CTRL_REG2_G  =  0x11
CTRL_REG3_G  =  0x12

CTRL_REG1_ALT = 0x20
CTRL_REG2_ALT = 0x21
CTRL_REG4_ALT = 0x23

GYRO_WHO_AM_I  = 0x0F
# end PMODNav definitions

In [ ]:
# testing SPI output

# SPI CORE config
IP_BASE_ADDRESS = 0x41E00000
ADDRESS_RANGE = 0xFFFF


# Address space definitions for PMODNAV
ACC_X_L_ADDR   =  0x28
MAG_X_L_ADDR   =  0x28
GYRO_X_L_ADDR  =  0x18
ALT_X_L_ADDR   =  0x28

CTRL_REG6_XL_ADDR  =  0x20

X_DIR_SEL    =   0x01
Y_DIR_SEL    =   0x02
Z_DIR_SEL    =   0x03

READ_CFG    =   0x80  # bit 7 high
WRITE_CFG   =   0x7F  # bit 7 low
MS_CFG      =   0xC0  # 11000000

STATE_ON    =   0x01
STATE_OFF   =   0x00

CTRL_REG5_XL =  0x1F
CTRL_REG6_XL =  0x20
CTRL_REG4    =  0x1E

CTRL_REG1_M  =  0x20
CTRL_REG2_M  =  0x21
CTRL_REG3_M  =  0x22
CTRL_REG4_M  =  0x23
CTRL_REG5_M  =  0x24

CTRL_REG1_G  =  0x10
CTRL_REG2_G  =  0x11
CTRL_REG3_G  =  0x12

CTRL_REG1_ALT = 0x20
CTRL_REG2_ALT = 0x21
CTRL_REG4_ALT = 0x23

GYRO_WHO_AM_I  = 0x0F
# end PMODNav definitions

# creating SPI CORE object for messages
SPI_CORE = MMIO(IP_BASE_ADDRESS,ADDRESS_RANGE)

# this is a basic configuration of the SPI core
# I would read through the docs a bit more and make sure this is correct
# it seems to work as is, 8 bit addres and 8 bit data with SS and CLK

# testing config
SPI_CORE.write(0x60, 0x006) # enable SPI core in master mode, auto SS
SPI_CORE.write(0x70, 0xFFFE) # enable SS 0



print('done')


In [ ]:
# testing write

OFFSET = 0x68

ADDRESS = 0X55
DATA = 0X55

# I put this together for a basic configuration of the messages
# each read/write needs to be tailored so
MESSAGE = (ADDRESS << 8) | DATA 

SPI_CORE.write(OFFSET, MESSAGE) # try to send data

#ADDRESS = 0XDE
#DATA = 0XAD

# I put this together for a basic configuration of the messages
# each read/write needs to be tailored so
#MESSAGE = (ADDRESS << 8) | DATA 

#SPI_CORE.write(OFFSET, MESSAGE) # try to send data

#ADDRESS = 0XBE
#DATA = 0XEF
#MESSAGE = (ADDRESS << 8) | DATA 

#SPI_CORE.write(OFFSET, MESSAGE) # try to send data

#print('done')